# Projeto 1 - Ciência dos Dados

Nome: Mariana Barbosa Sousa

Nome: Eduardo Papandrea Santana

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [19]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os


**Em `filename`, coloque o nome do seu arquivo de dados!**

In [20]:
import os

filename = 'redmi note 8.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo redmi note 8.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [21]:
#Base de treinamento:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,@rodrigofee2 garo realizar meu sonho pofavo me...,0
1,"me desculpem os demais, mais meu redmi note 8 ...",0
2,@infernkko foi o xiomi redmi note 8 pro miga,0
3,@xiaomi miui12 for redmi note 8?,0
4,"@nirtoc comprei o meu redmi note 8 ontem, como...",0


In [22]:
#Base de teste:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,@webernatali xiaomi. tenho um redmi note 8 pro...,1.0
1,"@pati_lima comprei o meu redmi note 8 ontem, c...",0.0
2,"testemunhas de xiaomi, ajudem a tia véia com o...",0.0
3,"a que bom, mas falando nisso cadê a a atualiza...",0.0
4,se alguém pensa em comprar um xiaomi redmi not...,1.0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto escolhido foi o celular "redmi note 8", da marca chinesa Xiaomi, um smartphone bastante relevante no mercado atual, dito por muitos especialistas como concorrente direto dos "iPhones" da Apple. Para a classificação, consideramos como relevantes aqueles tweets que descrevem qualidades do celular, em relação a câmera, processamento, atualizações de hardware (que é sempre um assunto debatido pelos usuários de aparelhos Xiaomi), além de avaliações sobre o custo do smartphone no mercado. A fim de tornar a avaliação valiosa para a empresa em que o projeto será utilizado, também constam como tweets relevantes aqueles que descrevem alguma crítica negativa ao celular, seja em relação ao desempenho de processamento no dia-a-dia, ou em relação, novamente, ao preço que se encontra aqui no Brasil. Os tweets que não apresentavam algum tipo de opinião explícita como as citadas anteriormente, foram desconsiderados para a classificação.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [23]:
#Categorizando as classes dos tweets da base de treinamento:
train['Relevância'] = train['Relevância'].astype('category')
train['Relevância'].cat.categories = ['No', 'Yes']
train['Relevância'] = pd.Categorical(train['Relevância'], categories=['No', 'Yes'], ordered=False)
train.head(5)

,Treinamento,Relevância
0,@rodrigofee2 garo realizar meu sonho pofavo me...,No
1,"me desculpem os demais, mais meu redmi note 8 ...",No
2,@infernkko foi o xiomi redmi note 8 pro miga,No
3,@xiaomi miui12 for redmi note 8?,No
4,"@nirtoc comprei o meu redmi note 8 ontem, como...",No


In [24]:
#Categorizando as classes dos tweets da base de teste:
test['Relevância'] = test['Relevância'].astype('category')
test['Relevância'].cat.categories = ['No', 'Yes']
test['Relevância'] = pd.Categorical(test['Relevância'], categories=['No', 'Yes'], ordered=False)
test.head(5)

,Teste,Relevância
0,@webernatali xiaomi. tenho um redmi note 8 pro...,Yes
1,"@pati_lima comprei o meu redmi note 8 ontem, c...",No
2,"testemunhas de xiaomi, ajudem a tia véia com o...",No
3,"a que bom, mas falando nisso cadê a a atualiza...",No
4,se alguém pensa em comprar um xiaomi redmi not...,Yes


**Limpeza dos Dados**

Vamos fazer uma limpeza de pontuações, nome de usuários e links de compra  e "rt", por exemplo, uma vez que não são relevantes para a nossa classficação, já que não influenciam um tweet a ser relevante ou não. Ademais, vamospadronizar o tamanho das letras dos tweets para que o programa não considere a mesma letra maiúscula e minúscula como diferentes. Logo, vamos deixá-las todas minúsculas.

In [25]:
import re
#Limpando pontuação:
def cleanup(text):
    punctuation = '[!-.:?;\n/]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
#Limpando nome de usuários, já que não são relevantes para nossa análise:
def limpeza_usuarios(tweet):
    tweet  =  re.sub(r"@\S+" , "" , tweet )
    return tweet
#Removendo link desnecessários:
def Limpeza_dados(instancia):
    instancia = re.sub(r"http\S+", "", instancia)
    return instancia
for i in range(400):
    train = train.replace(train['Treinamento'][i], limpeza_usuarios(train['Treinamento'][i]))
    train = train.replace(train['Treinamento'][i], Limpeza_dados(train['Treinamento'][i]))
    train = train.replace(train['Treinamento'][i], cleanup(train['Treinamento'][i]))
      
    #Removendo "rt":
    if train['Treinamento'][i][:2] == 'rt':
        train = train.replace(train['Treinamento'][i], train['Treinamento'][i][3:])
    #Tranformando todas as letras em minúsculas:
    train = train.replace(train['Treinamento'][i], train['Treinamento'][i].lower())
    
train.head(5)
    
    

,Treinamento,Relevância
0,garo realizar meu sonho pofavo meu sonho é te...,No
1,me desculpem os demais mais meu redmi note 8 p...,No
2,foi o xiomi redmi note 8 pro miga,No
3,miui12 for redmi note 8,No
4,comprei o meu redmi note 8 ontem como baixo a...,No


In [26]:
#Limpeza de dados da base de teste:
for i in range(121):
    test = test.replace(test['Teste'][i], limpeza_usuarios(test['Teste'][i]))
    test = test.replace(test['Teste'][i], Limpeza_dados(test['Teste'][i]))
    test = test.replace(test['Teste'][i], cleanup(test['Teste'][i]))
      
    #Removendo "rt":
    if test['Teste'][i][:2] == 'rt':
        test = test.replace(test['Teste'][i], test['Teste'][i][3:])
    #Tranformando todas as letras em minúsculas:
    test = test.replace(test['Teste'][i], test['Teste'][i].lower())
    
test.head(5)

,Teste,Relevância
0,xiaomi tenho um redmi note 8 pro faz um ano e...,Yes
1,comprei o meu redmi note 8 ontem como baixo a...,No
2,testemunhas de xiaomi ajudem a tia véia com o ...,No
3,a que bom mas falando nisso cadê a a atualizaç...,No
4,se alguém pensa em comprar um xiaomi redmi not...,Yes


In [27]:
#Separando dados Relevantes e Não Relevantes:
R_train = train.loc[train['Relevância'] == 'Yes',:]
Rc_train = train.loc[train['Relevância'] == 'No',:]


In [28]:
#Transformando os dados Relevantes em uma lista
lista_R_train = []
for i in R_train['Treinamento']:
    s = i.split()
    for j in s:
        lista_R_train.append(j)
lista_R_train

['qual',
 'o',
 'teu',
 'meu',
 'redmi',
 'note',
 '8',
 'dura',
 'q',
 'é',
 'uma',
 'beleza1',
 'dia',
 'e',
 'meio',
 'com',
 'uso',
 'moderado',
 'fácil',
 'eu',
 'tenho',
 'um',
 'redmi',
 'note',
 '8',
 'e',
 'não',
 'me',
 'arrependo',
 'nem',
 'um',
 'pouco',
 'de',
 'ter',
 'escolhido',
 'o',
 'celular',
 'da',
 'xiaomi',
 'a',
 'câmera',
 'é',
 'perfeita',
 'também',
 'meu',
 'maior',
 'medo',
 'minha',
 'irmã',
 'yasmin',
 'comprou',
 'o',
 'redmi',
 'note',
 '8',
 'e',
 'eu',
 'achei',
 'mtooo',
 'bom',
 '🥺',
 'kkkkkk',
 'eu',
 'uso',
 'um',
 'redmi',
 'note',
 '8',
 'pro',
 'recomendo',
 'o',
 'meu',
 'é',
 'um',
 'redmi',
 'note',
 '8',
 'pro',
 'e',
 'eu',
 'tenho',
 'muito',
 'esse',
 'problema',
 'tem',
 'muito',
 'efeito',
 'de',
 'embelezamento',
 'e',
 'mesmo',
 'eu',
 'desmarcando',
 'essa',
 'opção',
 'ainda',
 'fica',
 'assim',
 'a',
 'luz',
 'muito',
 'clara',
 'me',
 'deixando',
 'mais',
 'branco',
 'claro',
 'que',
 'eu',
 'continuo',
 'negro',
 'kkkkkk',
 'ma

In [29]:
#Separando os emojis de emojis e de palavras para a lista de Relevantes:
from nltk.tokenize import TweetTokenizer
for i in range(len(lista_R_train)):
    tweet_tokenizer = TweetTokenizer() 
    lista_R_train[i] = tweet_tokenizer.tokenize(lista_R_train[i])
lista_R_train


[['qual'],
 ['o'],
 ['teu'],
 ['meu'],
 ['redmi'],
 ['note'],
 ['8'],
 ['dura'],
 ['q'],
 ['é'],
 ['uma'],
 ['beleza', '1'],
 ['dia'],
 ['e'],
 ['meio'],
 ['com'],
 ['uso'],
 ['moderado'],
 ['fácil'],
 ['eu'],
 ['tenho'],
 ['um'],
 ['redmi'],
 ['note'],
 ['8'],
 ['e'],
 ['não'],
 ['me'],
 ['arrependo'],
 ['nem'],
 ['um'],
 ['pouco'],
 ['de'],
 ['ter'],
 ['escolhido'],
 ['o'],
 ['celular'],
 ['da'],
 ['xiaomi'],
 ['a'],
 ['câmera'],
 ['é'],
 ['perfeita'],
 ['também'],
 ['meu'],
 ['maior'],
 ['medo'],
 ['minha'],
 ['irmã'],
 ['yasmin'],
 ['comprou'],
 ['o'],
 ['redmi'],
 ['note'],
 ['8'],
 ['e'],
 ['eu'],
 ['achei'],
 ['mtooo'],
 ['bom'],
 ['🥺'],
 ['kkkkkk'],
 ['eu'],
 ['uso'],
 ['um'],
 ['redmi'],
 ['note'],
 ['8'],
 ['pro'],
 ['recomendo'],
 ['o'],
 ['meu'],
 ['é'],
 ['um'],
 ['redmi'],
 ['note'],
 ['8'],
 ['pro'],
 ['e'],
 ['eu'],
 ['tenho'],
 ['muito'],
 ['esse'],
 ['problema'],
 ['tem'],
 ['muito'],
 ['efeito'],
 ['de'],
 ['embelezamento'],
 ['e'],
 ['mesmo'],
 ['eu'],
 ['desmarcand

In [30]:
#Criando uma lista com a separação de emojis:
lista_R_train
lista_R_train_nova = []
for i in lista_R_train:
    for j in i:
        lista_R_train_nova.append(j)
lista_R_train_nova

['qual',
 'o',
 'teu',
 'meu',
 'redmi',
 'note',
 '8',
 'dura',
 'q',
 'é',
 'uma',
 'beleza',
 '1',
 'dia',
 'e',
 'meio',
 'com',
 'uso',
 'moderado',
 'fácil',
 'eu',
 'tenho',
 'um',
 'redmi',
 'note',
 '8',
 'e',
 'não',
 'me',
 'arrependo',
 'nem',
 'um',
 'pouco',
 'de',
 'ter',
 'escolhido',
 'o',
 'celular',
 'da',
 'xiaomi',
 'a',
 'câmera',
 'é',
 'perfeita',
 'também',
 'meu',
 'maior',
 'medo',
 'minha',
 'irmã',
 'yasmin',
 'comprou',
 'o',
 'redmi',
 'note',
 '8',
 'e',
 'eu',
 'achei',
 'mtooo',
 'bom',
 '🥺',
 'kkkkkk',
 'eu',
 'uso',
 'um',
 'redmi',
 'note',
 '8',
 'pro',
 'recomendo',
 'o',
 'meu',
 'é',
 'um',
 'redmi',
 'note',
 '8',
 'pro',
 'e',
 'eu',
 'tenho',
 'muito',
 'esse',
 'problema',
 'tem',
 'muito',
 'efeito',
 'de',
 'embelezamento',
 'e',
 'mesmo',
 'eu',
 'desmarcando',
 'essa',
 'opção',
 'ainda',
 'fica',
 'assim',
 'a',
 'luz',
 'muito',
 'clara',
 'me',
 'deixando',
 'mais',
 'branco',
 'claro',
 'que',
 'eu',
 'continuo',
 'negro',
 'kkkkkk',

In [31]:
#Transformando a nova lista de Relevantes em uma série:
serie_R_train = pd.Series(lista_R_train_nova)
serie_R_train

0         qual
1            o
2          teu
3          meu
4        redmi
         ...  
4374        já
4375       tbm
4376    famoso
4377       bbb
4378      kkkk
Length: 4379, dtype: object

In [32]:
#Calculando frequência de cada palavra Relevante:
tabela_R_train = serie_R_train.value_counts()
tabela_R_train


note            200
redmi           189
8               179
o               159
e               137
               ... 
atualizações      1
começo            1
replicar          1
cima              1
motoboy           1
Length: 1090, dtype: int64

In [33]:
#Transformando os dados Não Relevantes em uma lista
lista_Rc_train = []
for i in Rc_train['Treinamento']:
    s = i.split()
    for j in s:
        lista_Rc_train.append(j)
lista_Rc_train

['garo',
 'realizar',
 'meu',
 'sonho',
 'pofavo',
 'meu',
 'sonho',
 'é',
 'ter',
 'um',
 'xiaomi',
 'redmi',
 'note',
 '8',
 'pofavo',
 'se',
 'vc',
 'puder',
 'vem',
 'direct',
 'joaosbalopes',
 'me',
 'desculpem',
 'os',
 'demais',
 'mais',
 'meu',
 'redmi',
 'note',
 '8',
 'pro',
 'de',
 '64',
 'megapixels',
 'tmb',
 'faz',
 'esse',
 'milagre',
 'nas',
 'fotos',
 '😘😚',
 'que',
 'comece',
 'o',
 'recalque',
 '😁',
 'foi',
 'o',
 'xiomi',
 'redmi',
 'note',
 '8',
 'pro',
 'miga',
 'miui12',
 'for',
 'redmi',
 'note',
 '8',
 'comprei',
 'o',
 'meu',
 'redmi',
 'note',
 '8',
 'ontem',
 'como',
 'baixo',
 'a',
 'gcam',
 '⭐⭐⭐⭐⭐celular',
 'xiaomi',
 'redmi',
 'note',
 '8',
 'pro',
 'versão',
 'global',
 'azul',
 'por',
 'xiaomi',
 'via',
 'amazon',
 'redmi',
 'note',
 '8',
 'pro',
 'canim',
 'a',
 'miui',
 '12',
 'para',
 'o',
 'redmi',
 'note',
 '8',
 'é',
 'real',
 'ou',
 'imaginária',
 'será',
 'um',
 'holograma',
 'simmas',
 'vc',
 'viu',
 'o',
 'redmi',
 'note',
 '8',
 'redmi',
 'not

In [34]:
#Separando os emojis de emojis e de palavras para a lista de Não Relevantes:
for i in range(len(lista_Rc_train)):
    tweet_tokenizer = TweetTokenizer() 
    lista_Rc_train[i] = tweet_tokenizer.tokenize(lista_Rc_train[i])
lista_Rc_train

[['garo'],
 ['realizar'],
 ['meu'],
 ['sonho'],
 ['pofavo'],
 ['meu'],
 ['sonho'],
 ['é'],
 ['ter'],
 ['um'],
 ['xiaomi'],
 ['redmi'],
 ['note'],
 ['8'],
 ['pofavo'],
 ['se'],
 ['vc'],
 ['puder'],
 ['vem'],
 ['direct'],
 ['joaosbalopes'],
 ['me'],
 ['desculpem'],
 ['os'],
 ['demais'],
 ['mais'],
 ['meu'],
 ['redmi'],
 ['note'],
 ['8'],
 ['pro'],
 ['de'],
 ['64'],
 ['megapixels'],
 ['tmb'],
 ['faz'],
 ['esse'],
 ['milagre'],
 ['nas'],
 ['fotos'],
 ['😘', '😚'],
 ['que'],
 ['comece'],
 ['o'],
 ['recalque'],
 ['😁'],
 ['foi'],
 ['o'],
 ['xiomi'],
 ['redmi'],
 ['note'],
 ['8'],
 ['pro'],
 ['miga'],
 ['miui', '12'],
 ['for'],
 ['redmi'],
 ['note'],
 ['8'],
 ['comprei'],
 ['o'],
 ['meu'],
 ['redmi'],
 ['note'],
 ['8'],
 ['ontem'],
 ['como'],
 ['baixo'],
 ['a'],
 ['gcam'],
 ['⭐', '⭐', '⭐', 'celular'],
 ['xiaomi'],
 ['redmi'],
 ['note'],
 ['8'],
 ['pro'],
 ['versão'],
 ['global'],
 ['azul'],
 ['por'],
 ['xiaomi'],
 ['via'],
 ['amazon'],
 ['redmi'],
 ['note'],
 ['8'],
 ['pro'],
 ['canim'],
 ['a'],

In [35]:
#Criando uma lista com os emojis separados:
lista_Rc_train
lista_Rc_train_nova = []
for i in lista_Rc_train:
    for j in i:
        lista_Rc_train_nova.append(j)
lista_Rc_train_nova

['garo',
 'realizar',
 'meu',
 'sonho',
 'pofavo',
 'meu',
 'sonho',
 'é',
 'ter',
 'um',
 'xiaomi',
 'redmi',
 'note',
 '8',
 'pofavo',
 'se',
 'vc',
 'puder',
 'vem',
 'direct',
 'joaosbalopes',
 'me',
 'desculpem',
 'os',
 'demais',
 'mais',
 'meu',
 'redmi',
 'note',
 '8',
 'pro',
 'de',
 '64',
 'megapixels',
 'tmb',
 'faz',
 'esse',
 'milagre',
 'nas',
 'fotos',
 '😘',
 '😚',
 'que',
 'comece',
 'o',
 'recalque',
 '😁',
 'foi',
 'o',
 'xiomi',
 'redmi',
 'note',
 '8',
 'pro',
 'miga',
 'miui',
 '12',
 'for',
 'redmi',
 'note',
 '8',
 'comprei',
 'o',
 'meu',
 'redmi',
 'note',
 '8',
 'ontem',
 'como',
 'baixo',
 'a',
 'gcam',
 '⭐',
 '⭐',
 '⭐',
 'celular',
 'xiaomi',
 'redmi',
 'note',
 '8',
 'pro',
 'versão',
 'global',
 'azul',
 'por',
 'xiaomi',
 'via',
 'amazon',
 'redmi',
 'note',
 '8',
 'pro',
 'canim',
 'a',
 'miui',
 '12',
 'para',
 'o',
 'redmi',
 'note',
 '8',
 'é',
 'real',
 'ou',
 'imaginária',
 'será',
 'um',
 'holograma',
 'simmas',
 'vc',
 'viu',
 'o',
 'redmi',
 'note'

In [36]:
#Transformando a lista de Não Relevantes em uma série:
serie_Rc_train = pd.Series(lista_Rc_train_nova)
serie_Rc_train

0           garo
1       realizar
2            meu
3          sonho
4         pofavo
          ...   
3947          no
3948       redmi
3949        note
3950           8
3951         pro
Length: 3952, dtype: object

In [37]:
#Calculando frequência relativa de cada palavra Não Relevante:
tabela_Rc_train = serie_Rc_train.value_counts()
tabela_Rc_train

note        236
redmi       231
8           210
o           120
xiaomi       81
           ... 
rondonia      1
ruim          1
relaxa        1
goiania       1
budokai       1
Length: 1079, dtype: int64

In [38]:
#Tranformando a base de treinamento em uma lista:
lista_train = []
for i in range(400):
    s = train['Treinamento'][i].split()
    for j in s:
        lista_train.append(j)
lista_train


['garo',
 'realizar',
 'meu',
 'sonho',
 'pofavo',
 'meu',
 'sonho',
 'é',
 'ter',
 'um',
 'xiaomi',
 'redmi',
 'note',
 '8',
 'pofavo',
 'se',
 'vc',
 'puder',
 'vem',
 'direct',
 'joaosbalopes',
 'me',
 'desculpem',
 'os',
 'demais',
 'mais',
 'meu',
 'redmi',
 'note',
 '8',
 'pro',
 'de',
 '64',
 'megapixels',
 'tmb',
 'faz',
 'esse',
 'milagre',
 'nas',
 'fotos',
 '😘😚',
 'que',
 'comece',
 'o',
 'recalque',
 '😁',
 'foi',
 'o',
 'xiomi',
 'redmi',
 'note',
 '8',
 'pro',
 'miga',
 'miui12',
 'for',
 'redmi',
 'note',
 '8',
 'comprei',
 'o',
 'meu',
 'redmi',
 'note',
 '8',
 'ontem',
 'como',
 'baixo',
 'a',
 'gcam',
 '⭐⭐⭐⭐⭐celular',
 'xiaomi',
 'redmi',
 'note',
 '8',
 'pro',
 'versão',
 'global',
 'azul',
 'por',
 'xiaomi',
 'via',
 'amazon',
 'qual',
 'o',
 'teu',
 'meu',
 'redmi',
 'note',
 '8',
 'dura',
 'q',
 'é',
 'uma',
 'beleza1',
 'dia',
 'e',
 'meio',
 'com',
 'uso',
 'moderado',
 'fácil',
 'eu',
 'tenho',
 'um',
 'redmi',
 'note',
 '8',
 'e',
 'não',
 'me',
 'arrependo',
 

In [39]:
#Separando emojis de emojis e de palavras da base de treinamento total:
for i in range(len(lista_train)):
    tweet_tokenizer = TweetTokenizer() 
    lista_train[i] = tweet_tokenizer.tokenize(lista_train[i])
lista_train

[['garo'],
 ['realizar'],
 ['meu'],
 ['sonho'],
 ['pofavo'],
 ['meu'],
 ['sonho'],
 ['é'],
 ['ter'],
 ['um'],
 ['xiaomi'],
 ['redmi'],
 ['note'],
 ['8'],
 ['pofavo'],
 ['se'],
 ['vc'],
 ['puder'],
 ['vem'],
 ['direct'],
 ['joaosbalopes'],
 ['me'],
 ['desculpem'],
 ['os'],
 ['demais'],
 ['mais'],
 ['meu'],
 ['redmi'],
 ['note'],
 ['8'],
 ['pro'],
 ['de'],
 ['64'],
 ['megapixels'],
 ['tmb'],
 ['faz'],
 ['esse'],
 ['milagre'],
 ['nas'],
 ['fotos'],
 ['😘', '😚'],
 ['que'],
 ['comece'],
 ['o'],
 ['recalque'],
 ['😁'],
 ['foi'],
 ['o'],
 ['xiomi'],
 ['redmi'],
 ['note'],
 ['8'],
 ['pro'],
 ['miga'],
 ['miui', '12'],
 ['for'],
 ['redmi'],
 ['note'],
 ['8'],
 ['comprei'],
 ['o'],
 ['meu'],
 ['redmi'],
 ['note'],
 ['8'],
 ['ontem'],
 ['como'],
 ['baixo'],
 ['a'],
 ['gcam'],
 ['⭐', '⭐', '⭐', 'celular'],
 ['xiaomi'],
 ['redmi'],
 ['note'],
 ['8'],
 ['pro'],
 ['versão'],
 ['global'],
 ['azul'],
 ['por'],
 ['xiaomi'],
 ['via'],
 ['amazon'],
 ['qual'],
 ['o'],
 ['teu'],
 ['meu'],
 ['redmi'],
 ['note']

In [40]:
lista_train
lista_train_nova = []
for i in lista_train:
    for j in i:
        lista_train_nova.append(j)
lista_train_nova

['garo',
 'realizar',
 'meu',
 'sonho',
 'pofavo',
 'meu',
 'sonho',
 'é',
 'ter',
 'um',
 'xiaomi',
 'redmi',
 'note',
 '8',
 'pofavo',
 'se',
 'vc',
 'puder',
 'vem',
 'direct',
 'joaosbalopes',
 'me',
 'desculpem',
 'os',
 'demais',
 'mais',
 'meu',
 'redmi',
 'note',
 '8',
 'pro',
 'de',
 '64',
 'megapixels',
 'tmb',
 'faz',
 'esse',
 'milagre',
 'nas',
 'fotos',
 '😘',
 '😚',
 'que',
 'comece',
 'o',
 'recalque',
 '😁',
 'foi',
 'o',
 'xiomi',
 'redmi',
 'note',
 '8',
 'pro',
 'miga',
 'miui',
 '12',
 'for',
 'redmi',
 'note',
 '8',
 'comprei',
 'o',
 'meu',
 'redmi',
 'note',
 '8',
 'ontem',
 'como',
 'baixo',
 'a',
 'gcam',
 '⭐',
 '⭐',
 '⭐',
 'celular',
 'xiaomi',
 'redmi',
 'note',
 '8',
 'pro',
 'versão',
 'global',
 'azul',
 'por',
 'xiaomi',
 'via',
 'amazon',
 'qual',
 'o',
 'teu',
 'meu',
 'redmi',
 'note',
 '8',
 'dura',
 'q',
 'é',
 'uma',
 'beleza',
 '1',
 'dia',
 'e',
 'meio',
 'com',
 'uso',
 'moderado',
 'fácil',
 'eu',
 'tenho',
 'um',
 'redmi',
 'note',
 '8',
 'e',
 '

In [41]:
len(lista_train_nova)

8331

In [42]:
#Transformando a nova lista de treinamento em uma série:
serie_train = pd.Series(lista_train_nova)
serie_train

0           garo
1       realizar
2            meu
3          sonho
4         pofavo
          ...   
8326          no
8327       redmi
8328        note
8329           8
8330         pro
Length: 8331, dtype: object

In [43]:
tabela_train = serie_train.value_counts()
tabela_train

note           436
redmi          420
8              389
o              279
e              194
              ... 
doguinho         1
torno            1
maaas            1
muita            1
maravilhosa      1
Length: 1741, dtype: int64

In [44]:
prob_relevancia = train['Relevância'].value_counts(True, sort=False).round(4)
prob_relevante = prob_relevancia[0]
prob_nao_relevante = prob_relevancia[1]



In [45]:
#Calculando o total de palavras possíveis sem repetição na lista total da base de treinamento, para aplicar a suavização de Laplace:
lista_sem_repeticao = []
for i in lista_train_nova:
    if not i in lista_sem_repeticao:
        lista_sem_repeticao.append(i)
print(len(lista_sem_repeticao))

1741


In [46]:
print("Número de elementos na lista de Relevantes: {0}.\nNúmero de elementos na lista de Não Relevantes:{1}.".format(len(serie_R_train), len(serie_Rc_train)))

Número de elementos na lista de Relevantes: 4379.
Número de elementos na lista de Não Relevantes:3952.


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [47]:
#Calculando a probabilidade de cada frase da base de Teste ser relevante ou não:
#Calculando a probabilidade de a frase ser Relevante:
lista_probalidade_dado_R = []
for i in test['Teste']:
    P_frase_dado_R = 1
    i = i.split()
    for j in i:
        if j in tabela_R_train:
            c = tabela_R_train[j]        
        else:
            c = 0
        prob = (c + 1)/(len(serie_R_train) + len(lista_sem_repeticao)) #Suavização de Laplace -> (contagem + 1)/(tamanho_serie + tamanho_lista_sem_repeticao)
        P_frase_dado_R *= prob
    lista_probalidade_dado_R.append(P_frase_dado_R)
lista_probalidade_dado_R

  
        
   


[9.045239141154379e-52,
 8.374770202395484e-27,
 9.785008720075083e-149,
 1.4844112706766433e-41,
 3.8447158613383566e-139,
 1.4966154793418055e-37,
 8.966566414703783e-36,
 3.878479635371461e-105,
 1.1303442836293212e-44,
 2.851182227504814e-55,
 5.644463433761386e-57,
 1.6121812238513066e-18,
 1.5404966182324095e-100,
 1.161568629355773e-40,
 3.1157809483082305e-59,
 7.06318703657204e-69,
 5.604838365551654e-12,
 1.530768090227813e-65,
 2.905831998911153e-55,
 1.439803411724335e-25,
 3.3058680707486854e-21,
 1.7008146300693627e-14,
 4.968894205324058e-28,
 6.745111011557129e-15,
 2.4201777090399023e-64,
 4.844748748266091e-61,
 2.424652500268691e-37,
 1.1209132311442694e-65,
 4.1590386002267726e-35,
 1.809025309017858e-43,
 3.0704926165874013e-20,
 4.7967462177859215e-31,
 4.3003618323853794e-37,
 4.5082161813618704e-131,
 2.1335049605923335e-62,
 9.884752450524292e-74,
 1.2502067926691387e-36,
 1.4340431303453873e-94,
 9.679738577964775e-43,
 4.700477225551124e-21,
 1.83699261817500

In [48]:
#Calculando a probabilidade de a frase ser Relevante:
lista_probalidade_dado_Rc = []
for i in test['Teste']:
    P_frase_dado_Rc = 1
    i = i.split()
    for j in i:
        if j in tabela_Rc_train:
            c = tabela_Rc_train[j]        
        else:
            c = 0
        prob = (c + 1)/(len(serie_R_train) + len(lista_sem_repeticao)) #Suavização de Laplace
        P_frase_dado_Rc *= prob
    lista_probalidade_dado_Rc.append(P_frase_dado_Rc)
lista_probalidade_dado_Rc

[4.4188964687873325e-55,
 2.5905207245226766e-25,
 9.189237008166145e-151,
 4.2109527575763826e-43,
 3.715834911596892e-147,
 8.507137662047728e-39,
 4.1903388919163484e-35,
 2.0310441508733407e-109,
 3.147215937864019e-47,
 5.415600051794396e-59,
 9.775999471655123e-57,
 6.315754554079342e-19,
 3.6765656068123655e-105,
 9.414307660159795e-39,
 2.6354318361473563e-58,
 2.0845827736366007e-67,
 5.405323200031989e-12,
 6.81611931612161e-65,
 2.796967290672504e-59,
 3.262078590039701e-26,
 3.4834407559444483e-22,
 2.5171848235443088e-14,
 3.088381175436034e-29,
 8.357128960183276e-15,
 2.709251556082963e-62,
 1.480179954185587e-68,
 3.1853108255093147e-40,
 2.0302891396360301e-66,
 2.0997871880259768e-35,
 9.22395784145317e-44,
 1.7031892577738178e-19,
 1.508791843346703e-30,
 4.5487554887918366e-40,
 1.8596556991239564e-126,
 1.378524157605215e-58,
 1.298268087304052e-72,
 1.8147468232261442e-36,
 3.2543930820297025e-95,
 9.24855968242774e-42,
 9.175430344909456e-21,
 4.271943249716844e-

In [51]:
for i in range(121):
    P_R_dado_frase =  lista_probalidade_dado_R[i]*prob_relevante
    P_Rc_dado_frase = lista_probalidade_dado_Rc[i]*prob_nao_relevante
    
    if (P_R_dado_frase > P_Rc_dado_frase):
        test['Naive Bayes'][i] = 'Yes' 
    else:
        test['Naive Bayes'][i] = 'No'
        
test
    

    
    
    


,Teste,Relevância,Naive Bayes
0,xiaomi tenho um redmi note 8 pro faz um ano e...,Yes,Yes
1,comprei o meu redmi note 8 ontem como baixo a...,No,No
2,testemunhas de xiaomi ajudem a tia véia com o ...,No,Yes
3,a que bom mas falando nisso cadê a a atualizaç...,No,Yes
4,se alguém pensa em comprar um xiaomi redmi not...,Yes,Yes
...,...,...,...
116,se é pra enaltecer a câmera da xiaomi eu tô aq...,Yes,Yes
117,a xiaomi tá de sacanagem com os donos do re...,No,Yes
118,tem tutorial no you tube joga lá redmi note 8...,No,Yes
119,mais uma venda concluída muito obrigado pela c...,No,Yes


In [52]:
Acertos = 0 
for i in range(121):
    if test['Relevância'][i] == test['Naive Bayes'][i]:
        Acertos+=1
Acertos
print(Acertos/121)

0.6115702479338843


In [53]:
Acertos

74

In [54]:
pd.crosstab(test['Relevância'], test['Naive Bayes'], normalize = True)

Naive Bayes,No,Yes
Relevância,,
No,0.391667,0.341667
Yes,0.041667,0.225000


In [66]:
#Verificando a perfomance:
relevancia = test['Relevância'].value_counts(False, sort=False).round(4)
relevante = relevancia[0]
nao_relevante = relevancia[1]

verdadeiros_positivos = 0
falsos_positivos = 0
verdadeiros_negativos = 0
falsos_negativos = 0

for i in range(len(test['Relevância'])):
    if test['Relevância'][i] == 'No' and test['Naive Bayes'][i] == 'No':
        verdadeiros_negativos += 1
    elif test['Relevância'][i] == 'Yes' and test['Naive Bayes'][i] == 'No':
        falsos_negativos += 1
    elif test['Relevância'][i] == 'Yes' and test['Naive Bayes'][i] == 'Yes':
        verdadeiros_positivos += 1
    elif test['Relevância'][i] == 'No' and test['Naive Bayes'][i] == 'Yes':
        falsos_positivos += 1
        
print("Classificação\t\t\tProbabilidade ")
print("----------------------------------------------")
print("%s:\t\t%f" % ('Verdadeiros Positivos', verdadeiros_positivos/relevancia[0]))
print("%s:\t\t%f" % ('Verdadeiros Negativos', verdadeiros_negativos/relevancia[1]))
print("%s:\t\t%f" % ('Falsos Positivos', falsos_positivos/relevancia[1]))
print("%s:\t\t%f" % ('Falsos Negativos', falsos_negativos/relevancia[0]))

Classificação			Probabilidade 
----------------------------------------------
Verdadeiros Positivos:		0.306818
Verdadeiros Negativos:		1.468750
Falsos Positivos:		1.281250
Falsos Negativos:		0.056818


___
### Concluindo

Pela tabela de sentimentos e pelas comparações posteriores feitas para verificar a perfomance, o classificador mostrou ter uma boa perfomance, uma vez que obteve um acerto de mais de 50% quando comparado com a análise feita manualmente. Divergências entre as análises feitas manualmente e pelo classificador de Naive-Bayes podem decorrer de tweets que se mostaram confusos quanto à classificação, o que pode ter culminado em diferentes classificações para um mesmo tweet. Ademais, tweets que contêm sarcasmo e dulpa negação, por exemplo, podem ter sido classificados de forma errônea pelo classificador, uma vez que o programa não consegue entender a ideia por trás desse tipo de tweet. Apesar desses contrapontos, nosso classificador se mostrou bem proveitoso, uma vez que nosso cliente pode usá-lopara classificar os tweets sobre o redmi note 8 e não precisa se preocupar em ler todos os tweets, uma vez que nosso classificador tem uma grande taxa de acertos.
Entretanto, nosso classificador não pode ser usado para alimentar a base de treinamento automaticamente, pois como não haverá uma validação, pode haver classificações erradas e não terá com confirmar se o classificador clasificou certo ou não sem uma validação adequada. Além de ajudar a classificar tweets sobre o redmi note 8, o classificador de Naive Bayes pode ajudar a analisar  tweets relacionados a problemas de saúde, como depressão ou sobre o novo vírus, tweets que possam ser relevantes para resolução de casos policiais, por exemplo e tweets sobre a populariadade de alguém, geralmente alguma figura pública.
À luz dessas considerações, algumas melhorias qe poderiam ser feitas no nosso classificador seria remover palavras que não são relevantes para a classificação, como palavras comuns aos tweets, as "stopwords", que são preposições, por exemplo, e palavras com letras repetidas como "kkkk", que também podem ter causado problemas na classificação. Alguns sites que possam ajudar a nos basear para futuras iterações: https://towardsdatascience.com/creating-the-twitter-sentiment-analysis-program-in-python-with-naive-bayes-classification-672e5589a7ed; https://minerandodados.com.br/analise-de-sentimentos-utilizando-dados-do-twitter/

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**